<a href="https://colab.research.google.com/github/hhshanto/Dog-Breed-Identification/blob/main/Dog_Breed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# End to end multiclass dog breed classification
This Notebook builds an end to end multiclass image classifier using TensorFlow 2.0 and TensorFlow Hub

## 1. Problem
Identifying the breed of a dog given and image of a dog.
## 2. Data
The data is available on Kaggle:
https://www.kaggle.com/c/dog-breed-identification/data
## 3. Evaluation
Submissions are evaluated on **Multi Class Log Loss** between the predicted probability and the observed target.
## 4. Features
Info about data:
* 

# Get our workspace ready

In [8]:
 # Import tensorflow into Colab
 import tensorflow as tf
 print("TF version:", tf.__version__)

TF version: 2.5.0
